### Distributed Data Parallel Training

#### 本章内容适合观看B站up--你可是处女座的分布式训练视频 ####

#### 一、分布式通信的语言
1. Broadcast（广播）：一个 GPU 把数据发给所有人
2. All-Reduce（全规约）：所有人把自己的梯度求和，再把结果同步回给所有人。这是数据并行中最常用的操作
3. Reduce-Scatter（规约分发）：求和后，每个人只拿走一部分结果
4. All-Gather（全收集）：每个人把自己手里的那一小块数据发给所有人，最后大家都拥有一份完整的备份。
5. 核心公式：All-Reduce = Reduce-Scatter + All-Gather

#### 二、三大主流“切分刀法” (3D Parallelism)

1. 数据并行 (Data Parallelism, DP)
• 做法：每个 GPU 都存一份完整的模型，但处理不同的数据批次（Batch）
• 技术细节：在反向传播结束后，所有 GPU 通过 All-Reduce 同步梯度，确保大家更新后的权重是一致的。
• 优缺点：最简单、最通用，但当模型大到一张卡装不下时，它就失效了
1. 张量并行 (Tensor Parallelism, TP)
• 做法：把每一层内部的大矩阵“横着切”或“竖着切”，让不同的 GPU 协作完成一次矩阵乘法
• 技术细节：这需要极高的硬件互联速度（如 NVLink），通常只在单台机器内的 8 张显卡间进行。
• 示例：在 MLP 层中，GPU 1 计算前半部分神经元，GPU 2 计算后半部分，最后通过 All-Reduce 合并结果
1. 流水线并行 (Pipeline Parallelism, PP)
• 做法：把模型按层切开。比如 GPU 1 算前 10 层，GPU 2 算后 10 层
• 技术细节：为了减少“气泡”（即后面的卡等前面的卡吐数据），我们会使用 Micro-batches 像流水线一样连续

#### 三、显存节省神器：ZeRO 优化器 (FSDP)
即使有了上述并行，显存依然很紧缺。微软开发的 ZeRO (Zero Redundancy Optimizer) 技术（PyTorch 中称为 FSDP）彻底改变了现状：  

• Stage 1：只对优化器状态进行切片  

• Stage 2：对梯度也进行切片  

• Stage 3：对模型参数也进行切片  

• 原理：虽然我们在做数据并行，但每张卡只存自己负责的那一小块参数。计算时，通过 All-Gather “临时借用”别人的参数，算完立刻丢弃

#### 四、高级扩展
1. 专家并行 (Expert Parallelism)：专门针对混合专家模型 (MoE)。将不同的“专家”存放在不同的设备上，根据 Token 的需求进行路由转发
2. 序列并行/环形注意力 (Ring Attention)：为了处理超长文本，将 Attention 的计算也沿着序列长度维度切分到不同 GPU 上
3. 重计算 (Activation Checkpointing)：为了省显存，前向传播时不存激活值，反向传播时现场重算。这是一种典型的“以计算换空间”的策略